## Predicting Columns in a Table - Quick Start


Source: https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-quickstart.html 

Via a simple fit() call, AutoGluon can produce highly-accurate models to predict the values in one column of a data table based on the rest of the columns’ values. Use AutoGluon with tabular data for both classification and regression problems. This tutorial demonstrates how to use AutoGluon to produce a classification model that predicts whether or not a person’s income exceeds $50,000.

To start, import AutoGluon’s TabularPredictor and TabularDataset classes:

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

Load training data from a CSV file into an AutoGluon Dataset object. This object is essentially equivalent to a Pandas DataFrame and the same methods can be applied to both.

In [2]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()
train_data.to_csv('IncomePrediction.csv', index=False)

Note that we loaded data from a CSV file stored in the cloud (AWS s3 bucket), but you can you specify a local file-path instead if you have already downloaded the CSV file to your own machine (e.g., using wget). Each row in the table train_data corresponds to a single training example. In this particular dataset, each row corresponds to an individual person, and the columns contain various characteristics reported during a census.

Let’s first use these features to predict whether the person’s income exceeds $50,000 or not, which is recorded in the class column of this table.

In [3]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object


Now use AutoGluon to train multiple models:



In [4]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.4.1
Python Version:     3.8.13
Operating System:   Darwin
Train Data Rows:    500
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting Au

Next, load separate test data to demonstrate how to make predictions on new examples at inference time:

In [5]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


We use our trained models to make predictions on the new data and then evaluate performance:



In [6]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8374449790152523
Evaluations on test data:
{
    "accuracy": 0.8374449790152523,
    "balanced_accuracy": 0.7430558394221018,
    "mcc": 0.5243657567117436,
    "f1": 0.621904761904762,
    "precision": 0.69394261424017,
    "recall": 0.5634167385677308
}


Predictions:  
 0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


We can also evaluate the performance of each individual trained model on our (labeled) test data:

In [7]:
predictor.leaderboard(test_data, silent=True)


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842973,0.84,0.107695,0.038007,0.391090,0.107695,0.038007,0.391090,1,True,5
1,CatBoost,0.842461,0.85,0.015880,0.006318,0.611108,0.015880,0.006318,0.611108,1,True,7
2,RandomForestEntr,0.841130,0.83,0.094060,0.038199,0.374178,0.094060,0.038199,0.374178,1,True,6
3,LightGBM,0.839799,0.85,0.017172,0.005376,0.192741,0.017172,0.005376,0.192741,1,True,4
4,XGBoost,0.837445,0.87,0.031539,0.005458,0.303976,0.031539,0.005458,0.303976,1,True,11
5,WeightedEnsemble_L2,0.837445,0.87,0.033594,0.005892,0.604423,0.002055,0.000434,0.300447,2,True,14
6,LightGBMXT,0.836421,0.83,0.009415,0.005291,0.728723,0.009415,0.005291,0.728723,1,True,3
7,ExtraTreesGini,0.833453,0.82,0.083261,0.037825,0.358811,0.083261,0.037825,0.358811,1,True,8
8,ExtraTreesEntr,0.832839,0.81,0.098690,0.037909,0.355642,0.098690,0.037909,0.355642,1,True,9
9,LightGBMLarge,0.828949,0.83,0.028509,0.005546,1.616996,0.028509,0.005546,1.616996,1,True,13


Now you’re ready to try AutoGluon on your own tabular datasets! As long as they’re stored in a popular format like CSV, you should be able to achieve strong predictive performance with just 2 lines of code:

```python
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label=<variable-name>).fit(train_data=<file-name>)
```

Description of fit():

Here we discuss what happened during fit().

Since there are only two possible values of the class variable, this was a binary classification problem, for which an appropriate performance metric is accuracy. AutoGluon automatically infers this as well as the type of each feature (i.e., which columns contain continuous numbers vs. discrete categories). AutogGluon can also automatically handle common issues like missing data and rescaling feature values.

We did not specify separate validation data and so AutoGluon automatically choses a random training/validation split of the data. The data used for validation is seperated from the training data and is used to determine the models and hyperparameter-values that produce the best results. Rather than just a single model, AutoGluon trains multiple models and ensembles them together to ensure superior predictive performance.

By default, AutoGluon tries to fit various types of models including neural networks and tree ensembles. Each type of model has various hyperparameters, which traditionally, the user would have to specify. AutoGluon automates this process.

AutoGluon automatically and iteratively tests values for hyperparameters to produce the best performance on the validation data. This involves repeatedly training models under different hyperparameter settings and evaluating their performance. This process can be computationally-intensive, so fit() can parallelize this process across multiple threads (and machines if distributed resources are available). To control runtimes, you can specify various arguments in fit() as demonstrated in the subsequent In-Depth tutorial.

For tabular problems, fit() returns a Predictor object. For classification, you can easily output predicted class probabilities instead of predicted classes:



In [8]:
pred_probs = predictor.predict_proba(test_data_nolab)
pred_probs.head(5)

,<=50K,>50K
0,0.982107,0.017893
1,0.988337,0.011663
2,0.573505,0.426495
3,0.998272,0.001728
4,0.990299,0.009701


Besides inference, this object can also summarize what happened during fit.


In [9]:
results = predictor.fit_summary()


*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0               XGBoost       0.87       0.005458  0.303976                0.005458           0.303976            1       True         11
1   WeightedEnsemble_L2       0.87       0.005892  0.604423                0.000434           0.300447            2       True         14
2              LightGBM       0.85       0.005376  0.192741                0.005376           0.192741            1       True          4
3              CatBoost       0.85       0.006318  0.611108                0.006318           0.611108            1       True          7
4        NeuralNetTorch       0.85       0.010912  2.222347                0.010912           2.222347            1       True         12
5      RandomForestGini       0.84       0.038007  0.391090                0.038007           0.391090        

/Users/caihaocui/opt/miniconda3/envs/autoML/lib/python3.8/site-packages/autogluon/core/utils/plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


From this summary, we can see that AutoGluon trained many different types of models as well as an ensemble of the best-performing models. The summary also describes the actual models that were trained during fit and how well each model performed on the held-out validation data. We can view what properties AutoGluon automatically inferred about our prediction task:

In [10]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  binary
AutoGluon identified the following types of features:
('category', [])  : 7 | ['workclass', 'education', 'marital-status', 'occupation', 'relationship', ...]
('int', [])       : 6 | ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', ...]
('int', ['bool']) : 1 | ['sex']


AutoGluon correctly recognized our prediction problem to be a binary classification task and decided that variables such as age should be represented as integers, whereas variables such as workclass should be represented as categorical objects. The feature_metadata attribute allows you to see the inferred data type of each predictive variable after preprocessing (this is it’s raw dtype; some features may also be associated with additional special dtypes if produced via feature-engineering, e.g. numerical representations of a datetime/text column).

We can evaluate the performance of each individual trained model on our (labeled) test data:

In [11]:
predictor.leaderboard(test_data, silent=True)


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842973,0.84,0.091274,0.038007,0.391090,0.091274,0.038007,0.391090,1,True,5
1,CatBoost,0.842461,0.85,0.014571,0.006318,0.611108,0.014571,0.006318,0.611108,1,True,7
2,RandomForestEntr,0.841130,0.83,0.087639,0.038199,0.374178,0.087639,0.038199,0.374178,1,True,6
3,LightGBM,0.839799,0.85,0.023005,0.005376,0.192741,0.023005,0.005376,0.192741,1,True,4
4,XGBoost,0.837445,0.87,0.027927,0.005458,0.303976,0.027927,0.005458,0.303976,1,True,11
5,WeightedEnsemble_L2,0.837445,0.87,0.029470,0.005892,0.604423,0.001543,0.000434,0.300447,2,True,14
6,LightGBMXT,0.836421,0.83,0.016897,0.005291,0.728723,0.016897,0.005291,0.728723,1,True,3
7,ExtraTreesGini,0.833453,0.82,0.088991,0.037825,0.358811,0.088991,0.037825,0.358811,1,True,8
8,ExtraTreesEntr,0.832839,0.81,0.103103,0.037909,0.355642,0.103103,0.037909,0.355642,1,True,9
9,LightGBMLarge,0.828949,0.83,0.031570,0.005546,1.616996,0.031570,0.005546,1.616996,1,True,13


When we call predict(), AutoGluon automatically predicts with the model that displayed the best performance on validation data (i.e. the weighted-ensemble). We can instead specify which model to use for predictions like this:

In [12]:
predictor.predict(test_data, model='LightGBM')


0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object

Above the scores of predictive performance were based on a default evaluation metric (accuracy for binary classification). Performance in certain applications may be measured by different metrics than the ones AutoGluon optimizes for by default. If you know the metric that counts in your application, you should specify it as demonstrated in the next section.

## Maximizing predictive performance
Note: You should not call fit() with entirely default arguments if you are benchmarking AutoGluon-Tabular or hoping to maximize its accuracy! To get the best predictive accuracy with AutoGluon, you should generally use it like this:

In [13]:
time_limit = 60  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')
predictor.leaderboard(test_data, silent=True)

No path specified. Models will be saved in: "AutogluonModels/ag-20220522_001624/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels/ag-20220522_001624/"
AutoGluon Version:  0.4.1
Python Version:     3.8.13
Operating System:   Darwin
Train Data Rows:    500
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,0.900802,0.881867,0.152478,0.040154,6.793864,0.152478,0.040154,6.793864,1,True,3
1,CatBoost_BAG_L1,0.900744,0.892278,0.063676,0.039250,6.863643,0.063676,0.039250,6.863643,1,True,7
2,WeightedEnsemble_L2,0.897745,0.902811,2.042755,0.450649,28.089640,0.002880,0.001098,1.255093,2,True,14
3,LightGBM_BAG_L1,0.892347,0.866991,0.087833,0.031637,10.254106,0.087833,0.031637,10.254106,1,True,4
4,XGBoost_BAG_L1,0.891483,0.868006,0.185786,0.040786,6.366773,0.185786,0.040786,6.366773,1,True,11
5,RandomForestEntr_BAG_L1,0.886810,0.889011,0.090887,0.093487,0.461353,0.090887,0.093487,0.461353,1,True,6
6,RandomForestGini_BAG_L1,0.885092,0.887407,0.085065,0.097172,0.427251,0.085065,0.097172,0.427251,1,True,5
7,NeuralNetFastAI_BAG_L1,0.885055,0.868290,0.857207,0.063946,2.896790,0.857207,0.063946,2.896790,1,True,10
8,NeuralNetTorch_BAG_L1,0.882333,0.845114,0.755222,0.059100,9.412979,0.755222,0.059100,9.412979,1,True,12
9,ExtraTreesEntr_BAG_L1,0.880568,0.887681,0.162326,0.116575,0.481117,0.162326,0.116575,0.481117,1,True,9


This command implements the following strategy to maximize accuracy:

Specify the argument presets='best_quality', which allows AutoGluon to automatically construct powerful model ensembles based on stacking/bagging, and will greatly improve the resulting predictions if granted sufficient training time. The default value of presets is 'medium_quality_faster_train', which produces less accurate models but facilitates faster prototyping. With presets, you can flexibly prioritize predictive accuracy vs. training/inference speed. For example, if you care less about predictive performance and want to quickly deploy a basic model, consider using: presets=['good_quality_faster_inference_only_refit', 'optimize_for_deployment'].

- Provide the eval_metric if you know what metric will be used to evaluate predictions in your application. Some other non-default metrics you might use include things like: 'f1' (for binary classification), 'roc_auc' (for binary classification), 'log_loss' (for classification), 'mean_absolute_error' (for regression), 'median_absolute_error' (for regression). You can also define your own custom metric function, see examples in the folder: autogluon/core/metrics/

- Include all your data in train_data and do not provide tuning_data (AutoGluon will split the data more intelligently to fit its needs).

- Do not specify the hyperparameter_tune_kwargs argument (counterintuitively, hyperparameter tuning is not the best way to spend a limited training time budgets, as model ensembling is often superior). We recommend you only use hyperparameter_tune_kwargs if your goal is to deploy a single model rather than an ensemble.

- Do not specify hyperparameters argument (allow AutoGluon to adaptively select which models/hyperparameters to use).

- Set time_limit to the longest amount of time (in seconds) that you are willing to wait. AutoGluon’s predictive performance improves the longer fit() is allowed to run.

## Regression (predicting numeric table columns):
To demonstrate that fit() can also automatically handle regression tasks, we now try to predict the numeric age variable in the same table based on the other features:

In [14]:
age_column = 'age'
print("Summary of age variable: \n", train_data[age_column].describe())

Summary of age variable: 
 count    500.00000
mean      39.65200
std       13.52393
min       17.00000
25%       29.00000
50%       38.00000
75%       49.00000
max       85.00000
Name: age, dtype: float64


We again call fit(), imposing a time-limit this time (in seconds), and also demonstrate a shorthand method to evaluate the resulting model on the test data (which contain labels):

In [15]:
predictor_age = TabularPredictor(label=age_column, path="agModels-predictAge").fit(train_data, time_limit=60)
performance = predictor_age.evaluate(test_data)

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "agModels-predictAge/"
AutoGluon Version:  0.4.1
Python Version:     3.8.13
Operating System:   Darwin
Train Data Rows:    500
Train Data Columns: 14
Label Column: age
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (85, 17, 39.652, 13.52393)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    45755.64 MB
	Train Data (Original)  Memory Usage: 0.32 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify sp

Note that we didn’t need to tell AutoGluon this is a regression problem, it automatically inferred this from the data and reported the appropriate performance metric (RMSE by default). To specify a particular evaluation metric other than the default, set the eval_metric argument of fit() and AutoGluon will tailor its models to optimize your metric (e.g. eval_metric = 'mean_absolute_error'). For evaluation metrics where higher values are worse (like RMSE), AutoGluon may sometimes flips their sign and print them as negative values during training (as it internally assumes higher values are better).

We can call leaderboard to see the per-model performance:

In [16]:
predictor_age.leaderboard(test_data, silent=True)


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-10.496803,-11.208568,0.329039,0.050857,2.422818,0.003141,0.000465,0.341385,2,True,12
1,ExtraTreesMSE,-10.656025,-11.369094,0.076334,0.030038,0.336550,0.076334,0.030038,0.336550,1,True,7
2,RandomForestMSE,-10.745634,-11.666909,0.069041,0.030291,0.302995,0.069041,0.030291,0.302995,1,True,5
3,CatBoost,-10.780312,-11.799279,0.013576,0.006133,0.531256,0.013576,0.006133,0.531256,1,True,6
4,LightGBMXT,-10.837373,-11.709228,0.061092,0.005072,0.365491,0.061092,0.005072,0.365491,1,True,3
5,LightGBM,-10.972156,-11.929546,0.053501,0.005208,0.207865,0.053501,0.005208,0.207865,1,True,4
6,XGBoost,-11.115033,-12.289224,0.029044,0.005100,0.274129,0.029044,0.005100,0.274129,1,True,9
7,NeuralNetFastAI,-11.225699,-12.073282,0.127432,0.010440,0.508987,0.127432,0.010440,0.508987,1,True,8
8,NeuralNetTorch,-11.448391,-11.995427,0.159428,0.010182,1.105263,0.159428,0.010182,1.105263,1,True,10
9,LightGBMLarge,-11.469922,-12.315314,0.029304,0.005564,2.132121,0.029304,0.005564,2.132121,1,True,11


Data Formats: AutoGluon can currently operate on data tables already loaded into Python as pandas DataFrames, or those stored in files of CSV format or Parquet format. If your data live in multiple tables, you will first need to join them into a single table whose rows correspond to statistically independent observations (datapoints) and columns correspond to different features (aka. variables/covariates).

Refer to the TabularPredictor documentation to see all of the available methods/options.